In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
%cd ..

synth_count = 3

from domblar.domblar import Domblar
d = Domblar(
    synth_count=synth_count,
    context='dexed',
    analysis_mode=False,
)

/Users/gexahedron/devel/domblar


In [2]:
# Philip Glass - Spaceship
# exploring voice-leading, modulations,
# triadic chord progressions, tonnetz
# transformations, neo-riemannian theories
# contrary and similar motions
# 12edo for now

from domblar.transformations import transpose
# FIXME
# import domblar.transformations.neo_riemannian_theory as nrt
from domblar.edo import Scale

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'harpic3')
set_synths()


def gen_chord(edo, name):
    # FIXME: what do i even want to do here?
    chord = [0, 5, 8]  # 5/F minor
    return transpose(chord, 0)#edo)
    # return [2, 5, 9]  # 2/D minor


def gen_gesture(chord):
    # TODO: replace chord with something more generic
    # these are indexes of input notes
    # if there's a tuple - then add octaves

    # TODO: the chord could have additional info on
    # which note is root, which note is 3rd, etc.
    # could be useful for initial gesture creation

    # return [0, 1, 2]

    # Concerto Grosso
    return [0, 1, 2,
            (0, 1), 2,
            (0, 1), 2]


class ChordKB():
    def __init__(self, modality, edo, intervals, annotations=['root', '3rd', '5th']):
        assert len(intervals) == len(annotations)
        self.modality = modality
        self.edo = edo
        self.intervals = intervals
        # FIXME: there's python builtin __annotations__
        # which will be confusing in the future
        self.annotations = annotations
        self.build_chords()

    def build_chords(self):
        self.chords = dict()
        for note in range(edo):
            self.chords[note] = frozenset([(note + interval) % edo for interval in self.intervals])
        self.chords_set = set(self.chords.values())

    def __contains__(self, pc_set):
        return pc_set in self.chords_set

    def annotate_chord(self, chord, pc_set):
        # TODO: chord and pc_set are interrelated and shouldn't be independent variables here
        assert pc_set in self
        root = None
        # TODO: optimize this search
        for note in range(self.edo):
            if self.chords[note] == pc_set:
                root = note
                break
        assert root is not None

        positions = []
        for note in chord:
            pc1 = note % edo  # TODO: should be a function
            for idx, interval in enumerate(self.intervals):
                pc2 = (root + interval) % edo  # TODO: should be a function
                if pc1 == pc2:
                    positions.append(self.annotations[idx])
        return positions


def annotate_chord(chord, edo):
    assert edo == 12
    # TODO: fetch knowledge base of chords for corresponding edo
    # TODO: code is super convoluted and hacky, yikes, what a mess

    # TODO: this should be a separate function
    pc_set = set()
    for note in chord:
        pc_set.add(note % edo)

    all_chords = []
    all_chords.append(ChordKB('maj', edo, [0, 4, 7]))
    all_chords.append(ChordKB('min', edo, [0, 3, 7]))
    all_chords.append(ChordKB('dim', edo, [0, 3, 6]))
    all_chords.append(ChordKB('dom7no5', edo, [0, 4, 10], ['root', '3rd', '7th']))

    for chord_type in all_chords:
        if pc_set in chord_type:
            positions = chord_type.annotate_chord(chord, pc_set)
            return chord_type.modality, positions
    assert False


def generic_chord_transform(chord, edo, shifts):
    assert edo == 12
    modality, annotated_chord = annotate_chord(chord, edo)
    assert modality in shifts
    new_chord = []
    for idx, note in enumerate(chord):
        shift = 0
        if annotated_chord[idx] in shifts[modality]:
            shift = shifts[modality][annotated_chord[idx]]
        new_chord.append(note + shift)
    if 'expect' in shifts[modality]:
        new_modality, _ = annotate_chord(new_chord, edo)
        assert new_modality == shifts[modality]['expect']
    return new_chord


def p(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'3rd': -1, 'expect': 'min'},
        'min': {'3rd': 1, 'expect': 'maj'}
    })

# FIXME: I don't like this naming
def el(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'root': -1, 'expect': 'min'},
        'min': {'5th': 1, 'expect': 'maj'}
    })

def r(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'5th': 2, 'expect': 'min'},
        'min': {'root': -2, 'expect': 'maj'}
    })

def m(chord, edo):
    assert edo == 12
    # FIXME: calculating all these shifts is fragile, error-prone
    # we need to find a better way to code them
    # maybe through some kind of discovery process
    # and while building knowledge base
    # although we also need to attach names to this operations
    return generic_chord_transform(chord, edo, {
        'maj': {'3rd': -1, '5th': 1, 'expect': 'maj'},
        'min': {'root': -1, '5th': 1, 'expect': 'min'},
    })

def slide(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'root': 1, '5th': 1, 'expect': 'min'},
        'min': {'root': -1, '5th': -1, 'expect': 'maj'},
    })

def dim_wat1(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'3rd': 2, '5th': 2, 'expect': 'dim'},
    })

# TODO: gen_dom7_from_dim

def apply_gestures(chords, gestures, edo):
    result = []
    for idx, chord in enumerate(chords):
        motif = []
        for pos in gestures[idx]:
            # FIXME: i assume here that gesture output is melody
            # but it could also be chords
            if type(pos) is int:
                note_idx = pos
                octave = 0
            else:
                note_idx, octave = pos
            note = chord[note_idx] + octave * edo
            motif.append(note)
        result.extend(motif)
    return result


edo = 12
scale = list(range(edo))


# FIXME
chord0 = gen_chord(edo, 'maj')
# chord1 = nrt.p(chord, edo)
chord1 = el(chord0, edo)
chord2 = m(chord1, edo)
chord3 = dim_wat1(chord2, edo)
chord4 = slide(chord0, edo)
print(chord0, chord1, chord2, chord3, chord4)

# gesture = gen_gesture(chord)
gestures = [
    [1, 0, 1, 2],
    [0, 1, 2],
    [0, 1, 2, 1],
    [0, 1, 2],
    [1, 0, 1, 2],
]

chords = [chord0, chord1, chord2, chord3, chord4]
print(chords)
melody = apply_gestures(chords, gestures, edo)
print(melody)

bass = [
    5, 5, 5, ',',
    1, 1, '.',
    9-edo, 9-edo, 9-edo, '.',
    11-edo, 11-edo, '.',
    4, 4, 4, '.']
bass = transpose(bass, -edo)

voices = list(zip(melody, bass))

dur = 0.15
# d.play(voices, scale, edo,
#        synth_idx=[0, 1],
#        dur=dur, sus=dur * 0.95,
#        voice_amps=[1, 1, 1, 1, 0.4, 0.4])


[0, 5, 8] [1, 5, 8] [1, 4, 9] [3, 6, 9] [-1, 4, 8]
[[0, 5, 8], [1, 5, 8], [1, 4, 9], [3, 6, 9], [-1, 4, 8]]
[5, 0, 5, 8, 1, 5, 8, 1, 4, 9, 4, 3, 6, 9, 4, -1, 4, 8]


In [5]:
# Philip Glass - In the Upper Room, Dance 1
# NRT parts (that's more of an exercise that an analysis)

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'harpic3')
set_synths()

edo = 12
scale = list(range(edo))

def dim_wat2(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'min': {'root': -2, '3rd': 1, 'expect': 'dim'}
    })

def dim_to_dom7no5(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'dim': {'root': 1, '5th': 1, 'expect': 'dom7no5'}  # actually, SLIDE for maj is same movement
    })

def min_to_dom7no5(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'min': {'3rd': -1, '5th': -1, 'expect': 'dom7no5'}
    })

def dom7no5_to_dim(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'dom7no5': {'3rd': -1, '7th': -1, 'expect': 'dim'}  # same as min_to_dom7no5
    })

def f(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'maj': {'3rd': 1, '5th': 1, 'expect': 'min'},
        'min': {'3rd': 2, '5th': 2, 'expect': 'maj'}
    })

# same as p(r(p(...)))
# def oct_min3rd(chord, edo):
#     assert edo == 12
#     return generic_chord_transform(chord, edo, {
#         # 'maj': {'expect': 'min'},  # TODO
#         'min': {'root': 1, '3rd': 1, '5th': 2, 'expect': 'maj'}
#     })

# unused, but it's the obvious one
def min_to_dim1(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'min': {'5th': -1, 'expect': 'dim'}
    })

def min_to_dim2(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'min': {'5th': 2, 'expect': 'dim'}
    })

def dim_to_maj(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'dim': {'root': -2, '5th': 1, 'expect': 'maj'}
    })

def dim_to_min_wat(chord, edo):
    assert edo == 12
    return generic_chord_transform(chord, edo, {
        'dim': {'root': 1, '3rd': 1, '5th': 2, 'expect': 'min'}
    })

init_chord = [2, 9, 5]  # 2/D minor
# first stream
chords1 = []
chords1.append(init_chord)
chords1.append(chords1[-1])
chords1.append(dim_wat2(chords1[-1], edo))  # [0, 9, 6], 6/F# dim
chords1.append(dim_to_dom7no5(chords1[-1], edo))  # [1, 9, 7], 9/A dom7no5 (without 4/E, 5th)
chords1.append(init_chord)
chords1.append(chords1[-1])
chords1.append(dim_wat2(chords1[-1], edo))
chords1.append(dim_to_dom7no5(chords1[-1], edo))
chords1.append(init_chord)
chords1.append(chords1[-1])
chords1.append(min_to_dom7no5(chords1[-1], edo))  # [2, 8, 4], 4/E dom7no5 (without 11/B, 5th)
chords1.append(dom7no5_to_dim(chords1[-1], edo))  # [1, 7, 4], 1/C# dim = 4/E min6

chords1.append(p(init_chord, edo))  # [2, 9, 6], 2/D major
chords1.append(f(chords1[-1], edo))  # 7/G minor
chords1.append(f(chords1[-1], edo))  # 0+12/C major
chords1.append(f(chords1[-1], edo))  # 5+12/F minor
chords1.append(p(r(p(chords1[-1], edo), edo), edo))  # 2+12/D major
# alternative description from octatonic cycle (prprprpr)
# chords1.append(oct_min3rd(chords1[-1], edo))
chords1.append(f(chords1[-1], edo))  # 7/G minor
chords1.append(min_to_dim2(chords1[-1], edo))  # 4+12/E dim
chords1.append(chords1[-1])

# print(chords1)

# this branch is much nastier
chords2 = []
# chords2.append(init_chord)
# chords2.append(chords2[-1])
# chords2.append(dim_wat2(chords2[-1], edo))  # [0, 9, 6], 6/F# dim
# chords2.append(dim_to_maj(chords2[-1], edo))  # [1, 9, 4], 9/A major

chords2.append(p(init_chord, edo))  # [2, 9, 6], 2/D major
chords2.append(f(chords2[-1], edo))  # 7/G minor
chords2.append([0, 7, 4])  # 0/C major
chords2.append(f(chords2[-1], edo))  # 5+12/F minor
chords2.append([-3, 6, 0])  # 6/F# dim
chords2.append(dim_to_min_wat(chords2[-1], edo))  # 7/G minor
chords2.append([-2, 7, 0])  # 0/C dom7no3
chords2.append([4, 10, 7])  # 4/E dim
print(chords2)

dur = 1.25
d.play(chords2, scale, edo,
       synth_idx=[0, 1, 2],
       dur=dur, sus=dur * 0.95,
       voice_amps=[1, 1, 1, 1, 0.4, 0.4])


[[2, 9, 6], [2, 10, 7], [0, 7, 4], [0, 8, 5], [-3, 6, 0], [-2, 7, 2], [-2, 7, 0], [4, 10, 7]]


In [85]:
finetuning = 0

try:
    finetuning
except Exception as e:
    d.open_editor(0)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(0)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(0)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            d.load_preset(i)
    else:
        d.print_params(0)

In [ ]:
# cleanup
d.stop_server()